# Laboratorio 9
## Ataque y defensa de modelos de Deep Learning
Universidad del Valle de Guatemala<br>
Security Data Science<br>
Pablo Andrés Zamora Vásquez - 21780<br>
Diego Andrés Morales Aquino - 21762<br>

## Primera parte: Ataques

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from art.estimators.classification import TensorFlowV2Classifier
from art.attacks.evasion import FastGradientMethod
from sklearn.metrics import accuracy_score
import numpy as np

In [ ]:
# Modelo entrenado
model = keras.models.load_model("malware_classification_model.keras")

# Recompilar con eager mode habilitado
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"],
    run_eagerly=True  # Necesario para ART con TF2
)

c:\Users\Pablo Zamora\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [3]:
# Cargar datos
dataset_path = "malimg_paper_dataset_imgs"
img_height, img_width = 64, 64
batch_size = 32
seed = 42

datagen = ImageDataGenerator(rescale=1.0 / 255, validation_split=0.3)

val_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=seed
)

X_test, y_test = next(val_generator)
for _ in range(len(val_generator) - 1):
    x, y = next(val_generator)
    X_test = np.concatenate((X_test, x))
    y_test = np.concatenate((y_test, y))

Found 2790 images belonging to 25 classes.


In [4]:
# Envolver modelo con ART

classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=25,
    input_shape=(64, 64, 3),
    loss_object=tf.keras.losses.CategoricalCrossentropy(),
    clip_values=(0.0, 1.0)
)

### Ataque #1: FGSM (Evasión)

In [ ]:
# Ataque FGSM
attack = FastGradientMethod(estimator=classifier, eps=0.1)
X_adv = attack.generate(x=X_test)

In [ ]:
# Evaluación
preds_original = np.argmax(classifier.predict(X_test), axis=1)
preds_adv = np.argmax(classifier.predict(X_adv), axis=1)
true_labels = np.argmax(y_test, axis=1)

print("Accuracy original:", accuracy_score(true_labels, preds_original))
print("Accuracy con ataque FGSM:", accuracy_score(true_labels, preds_adv))

Accuracy original: 0.9594982078853047
Accuracy con ataque FGSM: 0.23225806451612904


### Ataque #2: Black-box (Inferencia)

In [16]:
# Reutilizar el mismo ImageDataGenerator pero con subset="training"
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=False,
    seed=seed
)

# Convertir train_generator en arrays
X_train, y_train = next(train_generator)
for _ in range(len(train_generator) - 1):
    x, y = next(train_generator)
    X_train = np.concatenate((X_train, x))
    y_train = np.concatenate((y_train, y))


Found 6549 images belonging to 25 classes.


In [ ]:
from art.attacks.inference.membership_inference import MembershipInferenceBlackBox
import numpy as np

# 1) Balancear tamaños
min_len = min(len(X_train), len(X_test))
X_train_bal, y_train_bal = X_train[:min_len], y_train[:min_len]
X_test_bal,  y_test_bal  = X_test[:min_len],  y_test[:min_len]

# 2) Instanciar el ataque
mi_attack = MembershipInferenceBlackBox(
    estimator=classifier,
    input_type="prediction",      # usa directamente las probabilidades del modelo
    attack_model_type="nn",       
    scaler_type="minmax",         # normaliza las features antes de entrenar el adversario
    nn_model_epochs=50,           
    nn_model_batch_size=32
)

# 3) Entrenar el ataque
mi_attack.fit(
    x=X_train_bal,
    y=y_train_bal,
    test_x=X_test_bal,
    test_y=y_test_bal
)

# 4) Inferir membership (pasando siempre las etiquetas)
pred_train = mi_attack.infer(x=X_train_bal, y=y_train_bal)
pred_test  = mi_attack.infer(x=X_test_bal,  y=y_test_bal)

# 5) Métricas
tpr       = np.mean(pred_train == 1)
fpr       = np.mean(pred_test  == 1)
advantage = tpr - fpr

print(f"TPR (miembros):    {tpr:.3f}")
print(f"FPR (no-miembros): {fpr:.3f}")
print(f"Advantage:      {advantage:.3f}")


TPR (miembros):    1.000
FPR (no-miembros): 0.513
Advantage:      0.487


## Segunda parte: Defensas

### Defensa #1: Adversarial Training

Se reentrena el modelo incluyendo ejemplos FGSM para que aprenda a ser robusto

In [ ]:
# Generar adversariales sobre tu train set
fgsm = FastGradientMethod(estimator=classifier, eps=0.1)
X_train_adv = fgsm.generate(x=X_train)
y_train_adv = y_train.copy()

#Combinar datos limpios + adversariales
X_comb = np.concatenate([X_train, X_train_adv], axis=0)
y_comb = np.concatenate([y_train, y_train_adv], axis=0)

# Clonar y recompilar tu modelo base
model_def = keras.models.clone_model(model)
model_def.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"],
    run_eagerly=True
)

# Reentrenar sobre el conjunto combinado
model_def.fit(
    X_comb, y_comb,
    epochs=10,
    batch_size=32,
    validation_split=0.2
)

# Envolver el modelo defendido
classifier_def = TensorFlowV2Classifier(
    model=model_def,
    nb_classes=25,
    input_shape=(64,64,3),
    loss_object=tf.keras.losses.CategoricalCrossentropy(),
    clip_values=(0.0,1.0)
)

# Evaluar defensa ante FGSM
X_adv_test = fgsm.generate(x=X_test)
acc_before = np.mean(np.argmax(model.predict(X_adv_test),axis=1) == np.argmax(y_test,axis=1))
acc_after  = np.mean(np.argmax(model_def.predict(X_adv_test),axis=1) == np.argmax(y_test,axis=1))
print(f"Accuracy FGSM antes de defensa: {acc_before:.3f}")
print(f"Accuracy FGSM tras adversarial training: {acc_after:.3f}")

Epoch 1/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 54s 161ms/step - accuracy: 0.4392 - loss: 1.8605 - val_accuracy: 0.4653 - val_loss: 1.2454
Epoch 2/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 52s 159ms/step - accuracy: 0.7892 - loss: 0.6568 - val_accuracy: 0.5302 - val_loss: 1.2035
Epoch 3/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 53s 162ms/step - accuracy: 0.8907 - loss: 0.3808 - val_accuracy: 0.5504 - val_loss: 1.2297
Epoch 4/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 54s 164ms/step - accuracy: 0.9237 - loss: 0.2480 - val_accuracy: 0.5553 - val_loss: 1.4057
Epoch 5/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 55s 166ms/step - accuracy: 0.9321 - loss: 0.2053 - val_accuracy: 0.5588 - val_loss: 1.7329
Epoch 6/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 53s 161ms/step - accuracy: 0.9433 - loss: 0.1683 - val_accuracy: 0.5553 - val_loss: 1.3384
Epoch 7/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 53s 160ms/step - accuracy: 0.9425 - loss: 0.1669 - val_accuracy: 0.5450 - val_loss: 1.8970
Epoch 8/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 53s 161ms/step - accuracy: 0.9528 - loss: 0

Bajo ataque FGSM el modelo original solo acertaba el 23.2 % de las imágenes adversariales. Tras reentrenar con ejemplos FGSM, el mismo adversario logra solo un 83.5 % de acierto, recuperando robustez y subiendo la accuracy en adversariales desde 0.23 hasta 0.84. Esto significa que adversarial training es muy efectivo atacando directamente la perturbación FGSM, pues el modelo aprendió a reconocer y resistir esas pequeñas modificaciones.

### Defensa 2: Feature Squeezing

Aplicar un preprocesamiento que reduce la cantidad de “bits” de información de cada píxel, haciendo más difícil ocultar perturbaciones pequeñas.

In [ ]:
from art.defences.preprocessor import FeatureSqueezing

# 1) Crear el preprocesador
squeezer = FeatureSqueezing(
    clip_values=(0.0, 1.0),
    bit_depth=2
)

# 2) Aplicar el preprocesador a los adversariales
X_squeezed, _ = squeezer(X_adv_test, y=y_test)

# 3) Evaluar
acc_clean    = np.mean(
    np.argmax(model.predict(X_squeezed),    axis=1) 
    == np.argmax(y_test, axis=1)
)
acc_defended = np.mean(
    np.argmax(model_def.predict(X_squeezed), axis=1) 
    == np.argmax(y_test, axis=1)
)

print(f"Accuracy con Feature Squeezing (modelo original): {acc_clean:.3f}")
print(f"Accuracy con Feature Squeezing (modelo defendido): {acc_defended:.3f}")

88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step
Accuracy con Feature Squeezing (modelo original): 0.383
Accuracy con Feature Squeezing (modelo defendido): 0.935


Aplicando el preprocesador de bit-depth=2 sobre el modelo sin retraining, la accuracy ante FGSM sube del 23.2% a 38.3%. Si se combina el squeezing con el modelo adversarialmente entrenado, la accuracy salta hasta 93.5 % sobre esos mismos ejemplos adversariales. Esto indica que Feature Squeezing, al reducir la información por píxel, ya aporta cierta protección al modelo original y refuerza aún más al modelo entrenado con adversariales.